# Tweet Sentiment Analyzer
Made by: Rehan Adi Satrya - 13518061

Tujuan dari aplikasi ini adalah agar pengguna dapat mengetahui apakah ketikan mereka pada media sosial berpotensi menyakiti seseorang

In [17]:
#Import Library yang dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf

from numpy import array
from sklearn.model_selection import train_test_split

from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [18]:
# Load Dataset
data = pd.read_csv('dataset_two.csv', index_col=0)
data = data[['Instagram Comment Text', 'Sentiment']]
data.head()

,Instagram Comment Text,Sentiment
Id,,
1,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,negative
2,Geblek lo tata...cowo bgt dibela2in balikan......,negative
3,Kmrn termewek2 skr lengket lg duhhh kok labil ...,negative
4,"Intinya kalau kesel dengan ATT nya, gausah ke ...",negative
5,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",negative


In [19]:
y = [0 if i=='negative' else 1 for i in data['Sentiment']]
x = data['Instagram Comment Text']

In [64]:
# Split Data

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

len(y_test)

120

In [21]:
(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[ 0 61]
 [ 1 59]]


In [22]:
# Model Configuration

vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [23]:
# Preprocessing - Tokenization

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(X_train)
testing_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding='post', truncating='post')
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding='post', truncating='post')

# COnvert to Numpy Array
training_padded = np.array(training_padded)
training_labels = np.array(y_train)

testing_padded = np.array(testing_padded)
testing_labels = np.array(y_test)

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

In [72]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, y_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(y_test, prediction_linear, output_dict=True)
# print(report)
print('positive: ', report['1'])
print('negative: ', report['0'])

Training time: 0.015003s; Prediction time: 0.003581s
positive:  {'precision': 0.574468085106383, 'recall': 0.421875, 'f1-score': 0.4864864864864865, 'support': 64}
negative:  {'precision': 0.4931506849315068, 'recall': 0.6428571428571429, 'f1-score': 0.5581395348837208, 'support': 56}


In [73]:
import re
def preprocessing(string):
  # Jadikan lowercase semua
  string = string.lower()
  # mention
  string = re.sub("@[A-Za-z0-9]+", '', string)
  # Hapus link dari tweet
  string = re.sub(r'https?://\S+', '', string)
  # Hapus non letter seperti emoji
  string = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', string)
  # Hashtag
  string = re.sub('#|_', '', string)
  return string

dor = preprocessing("hahahaha @redi aneh bangettt #hehe https://www.google.com")
print(dor)

hahahaha  aneh bangettt hehe 


In [83]:
tweet = """ ini orang omongannya gak bisa dijaga kali ya @akunbuzzer """
# tweet = """ halo @amandaeliora apa kabar """
tweet = preprocessing(tweet)
review_vector = vectorizer.transform([tweet]) # vectorizing
if(classifier_linear.predict(review_vector)==[1]):
  print("Sentimen positif")
else:
  print("Sentimen negatif")

Sentimen negatif
